In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import joblib
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Attention, Concatenate
from tensorflow.keras.models import Model

In [3]:
# Lớp Seq2Seq với Attention
class Seq2SeqAttention(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_size, **kwargs):
        super(Seq2SeqAttention, self).__init__(**kwargs)

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.encoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.decoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.attention = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(vocab_size, activation="softmax")

        # Lưu tham số
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size

    def call(self, inputs):
        encoder_inputs, decoder_inputs = inputs

        encoder_embedded = self.embedding(encoder_inputs)
        encoder_outputs, state_h, state_c = self.encoder_lstm(encoder_embedded)

        decoder_embedded = self.embedding(decoder_inputs)
        decoder_outputs, _, _ = self.decoder_lstm(decoder_embedded, initial_state=[state_h, state_c])

        # Áp dụng Attention
        context_vector = self.attention([decoder_outputs, encoder_outputs, encoder_outputs])
        concat_outputs = tf.concat([decoder_outputs, context_vector], axis=-1)

        outputs = self.dense(concat_outputs)
        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "hidden_size": self.hidden_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)



In [4]:
save_dir = "/content/drive/MyDrive/model_build/saved_model"

# Load tokenizer
tokenizer_path = os.path.join(save_dir, "tokenizer.pkl")
tokenizer = joblib.load(tokenizer_path)

# Load mô hình
model = tf.keras.models.load_model(
    os.path.join(save_dir, "seq2seq_model.keras"),
    custom_objects={"Seq2SeqAttention": Seq2SeqAttention}
)

print(" Mô hình & tokenizer đã được load thành công!")

 Mô hình & tokenizer đã được load thành công!


In [5]:
def generate_response(prompt, tokenizer, model, max_length=150):
    # Token hóa prompt đầu vào
    encoded_prompt = tokenizer(prompt, return_tensors="tf", padding=True, truncation=True, max_length=max_length)
    input_ids = encoded_prompt["input_ids"]
    attention_mask = encoded_prompt["attention_mask"]

    # Kiểm tra tokenizer có bos/eos token không
    if tokenizer.bos_token_id is not None:
        start_token = tokenizer.bos_token_id
    else:
        start_token = tokenizer.cls_token_id  # Dùng [CLS] nếu không có bos_token_id

    if tokenizer.eos_token_id is not None:
        end_token = tokenizer.eos_token_id
    else:
        end_token = tokenizer.sep_token_id  # Dùng [SEP] nếu không có eos_token_id

    # Khởi tạo decoder với token bắt đầu
    decoder_input = tf.convert_to_tensor([[start_token]], dtype=tf.int32)
    response_ids = []

    for _ in range(max_length):
        # Dự đoán từ tiếp theo
        predictions = model([input_ids, decoder_input], training=False)
        predicted_id = tf.argmax(predictions[:, -1, :], axis=-1).numpy()[0]

        if predicted_id == end_token:
            break

        response_ids.append(predicted_id)
        decoder_input = tf.concat([decoder_input, tf.convert_to_tensor([[predicted_id]], dtype=tf.int32)], axis=1)

    # Giải mã output
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    return response_text


In [6]:
prompt = "có được bán đất nông nghiệp không?"

In [7]:

response = generate_response(prompt, tokenizer, model)

print("🚀 Bot trả lời:", response)

🚀 Bot trả lời: Không, trừ trường hợp nhất mục đích sử dụng đất.
